# MLFlow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

## Interacting with the MLflow tracking server

The <code>MLflowClient</code> object allows us to interact with
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments() # client.list_experiments() deprecated in MLflow 2.14.3

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1721811618075, experiment_id='1', last_update_time=1721811618075, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721811523739, experiment_id='0', last_update_time=1721811523739, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
client.create_experiment(name="my-cool-experiment")

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 5.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: b94c80e5f64940e6b8ef824165cfa8ad, rmse: 5.2010
run id: 71b51db6d6494e61b544a684c3da93d6, rmse: 5.2093
run id: ddd8af4b56cb4df8ba669e4c6f022b32, rmse: 5.2093
run id: 14184b2c9f00424f8e0d3e3dd5699cf9, rmse: 5.2107
run id: 1adc8805b550418aadb95d52bd7fcb5c, rmse: 5.2184


## Interacting with the Model Registry
In this section We will use the <code>MlflowClient</code> instance to:

1. Register a new version for the experiment <code>nyc-taxi-regressor</code>
2. Retrieve the latests versions of the model <code>nyc-taxi-regressor</code>.
3. Transition the version <code>1</code> and <code>2</code> to "Staging" and adding annotations to it.

In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [6]:
run_id = "71b51db6d6494e61b544a684c3da93d6"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1721828073317, current_stage='None', description=None, last_updated_timestamp=1721828073317, name='nyc-taxi-regressor', run_id='71b51db6d6494e61b544a684c3da93d6', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/71b51db6d6494e61b544a684c3da93d6/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [10]:
run_id = "b94c80e5f64940e6b8ef824165cfa8ad"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1721830589709, current_stage='None', description=None, last_updated_timestamp=1721830589709, name='nyc-taxi-regressor', run_id='b94c80e5f64940e6b8ef824165cfa8ad', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b94c80e5f64940e6b8ef824165cfa8ad/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [11]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


/tmp/ipykernel_4320/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [12]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_4320/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1721828073317, current_stage='Staging', description=None, last_updated_timestamp=1721830614074, name='nyc-taxi-regressor', run_id='71b51db6d6494e61b544a684c3da93d6', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/71b51db6d6494e61b544a684c3da93d6/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [13]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_4320/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1721830589709, current_stage='Staging', description=None, last_updated_timestamp=1721830618908, name='nyc-taxi-regressor', run_id='b94c80e5f64940e6b8ef824165cfa8ad', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b94c80e5f64940e6b8ef824165cfa8ad/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1721830589709, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-07-24', last_updated_timestamp=1721830626643, name='nyc-taxi-regressor', run_id='b94c80e5f64940e6b8ef824165cfa8ad', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b94c80e5f64940e6b8ef824165cfa8ad/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [26]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging


/tmp/ipykernel_4320/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [27]:
model_version = 2
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_4320/3059153048.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1721830589709, current_stage='Production', description='The model version 2 was transitioned to Staging on 2024-07-24', last_updated_timestamp=1721831790463, name='nyc-taxi-regressor', run_id='b94c80e5f64940e6b8ef824165cfa8ad', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b94c80e5f64940e6b8ef824165cfa8ad/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [28]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1721830589709, current_stage='Production', description='The model version 2 was transitioned to Production on 2024-07-24', last_updated_timestamp=1721831815338, name='nyc-taxi-regressor', run_id='b94c80e5f64940e6b8ef824165cfa8ad', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b94c80e5f64940e6b8ef824165cfa8ad/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [ ]:
client.create_registered_model("sk-learn-random-forest-reg-model")
result = client.create_model_version(
    name="sk-learn-random-forest-reg-model",
    source="mlruns/0/d16076a3ec534311817565e6527539c0/artifacts/sklearn-model",
    run_id="d16076a3ec534311817565e6527539c0",
)

In [31]:
client.create_registered_model("nyc-taxi-regressor-staging")
result = client.create_model_version(
    name="nyc-taxi-regressor-staging",
    source="mlruns/1/71b51db6d6494e61b544a684c3da93d6/artifacts/models_mlflow",
    run_id="71b51db6d6494e61b544a684c3da93d6",
)

In [32]:
client.set_registered_model_tag("nyc-taxi-regressor-staging", "task", "regression")

In [33]:
client.set_registered_model_alias("nyc-taxi-regressor-staging", "champion", 1)

In [37]:
client.delete_model_version(name="nyc-taxi-regressor-staging", version=2)

In [38]:
client.create_registered_model("nyc-taxi-regressor-prod")
result = client.create_model_version(
    name="nyc-taxi-regressor-prod",
    source="mlruns/1/b94c80e5f64940e6b8ef824165cfa8ad/artifacts/models_mlflow",
    run_id="b94c80e5f64940e6b8ef824165cfa8ad",
)

In [ ]:
client.set_registered_model_tag("nyc-taxi-regressor-prod", "task", "regression")

In [ ]:
client.set_registered_model_alias("nyc-taxi-regressor-prod", "champion", 1)

## Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.
   
**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [20]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    categorical = ['PULocationID', 'DOLocationID']
    train_dicts = df[categorical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [21]:
df = read_dataframe("../data/yellow_tripdata_2023-03.parquet")

In [22]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [23]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [24]:
X_test = preprocess(df, dv)

In [25]:
target = "duration"
y_test = df[target].values

In [29]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 6min 39s, sys: 514 ms, total: 6min 40s
Wall time: 3min 35s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.668900352214225}

In [30]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 2min 1s, sys: 125 ms, total: 2min 1s
Wall time: 1min 4s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.67962618493662}

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

In [45]:
import mlflow.pyfunc

model_name = "nyc-taxi-regressor-staging"
alias = "champion"

champion_version = mlflow.xgboost.load_model(f"models:/{model_name}@{alias}")

In [46]:
champion_version

In [43]:
import xgboost as xgb

xgb_test = xgb.DMatrix(X_test, label=y_test)

In [47]:
y_pred = champion_version.predict(xgb_test)
mean_squared_error(y_test, y_pred, squared=False)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.67962618493662